# Chapter 5: Support Vector Machines

Bab ini memperkenalkan **Support Vector Machines (SVMs)**, salah satu model Machine Learning yang sangat kuat dan serbaguna. SVM mampu melakukan klasifikasi linear maupun non-linear, regresi, dan bahkan deteksi outlier. SVM sangat cocok untuk dataset berukuran kecil hingga menengah yang kompleks.

Ide fundamental di balik SVM adalah menemukan "jalan" terlebar (disebut *margin*) yang memisahkan kelas-kelas yang berbeda. Ini disebut **large margin classification**.

## Linear SVM Classification

SVM bekerja dengan membuat sebuah *decision boundary* yang tidak hanya memisahkan dua kelas, tetapi juga menjaga jarak sejauh mungkin dari *instance* pelatihan terdekat. *Instance* yang berada di tepi *margin* ini disebut **support vectors**, karena mereka yang "mendukung" atau menentukan *decision boundary*.

### Soft Margin vs. Hard Margin Classification
* **Hard Margin:** Mengharuskan semua *instance* berada di luar *margin* dan di sisi yang benar. Ini hanya berfungsi jika data dapat dipisahkan secara linear dan sangat sensitif terhadap *outlier*.
* **Soft Margin:** Pendekatan yang lebih fleksibel di mana kita mencari keseimbangan antara menjaga *margin* selebar mungkin dan membatasi *margin violations* (yaitu, *instance* yang berakhir di dalam *margin* atau bahkan di sisi yang salah). Keseimbangan ini dikontrol oleh *hyperparameter* `C`.

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

# Memuat dataset iris
iris = datasets.load_iris()
X = iris["data"][:, (2, 3)]  # petal length, petal width
y = (iris["target"] == 2).astype(np.float64)  # Iris virginica

# Membuat pipeline untuk penskalaan dan klasifikasi SVM
svm_clf = Pipeline([
        ("scaler", StandardScaler()),
        ("linear_svc", LinearSVC(C=1, loss="hinge", random_state=42))
    ])

svm_clf.fit(X, y)

# Membuat prediksi
svm_clf.predict([[5.5, 1.7]])

Penting untuk melakukan penskalaan fitur (*feature scaling*) sebelum menggunakan SVM, karena SVM sensitif terhadap skala fitur yang berbeda-beda.

## Nonlinear SVM Classification

Banyak dataset yang tidak dapat dipisahkan secara linear. Salah satu pendekatan untuk menangani data non-linear adalah dengan menambahkan lebih banyak fitur, seperti fitur polinomial. Namun, pendekatan yang lebih kuat dan ajaib pada SVM adalah dengan menggunakan **kernel trick**.

### Polynomial Kernel

Menambahkan fitur polinomial dapat membuat dataset yang tidak linear menjadi dapat dipisahkan secara linear. Namun, ini bisa sangat lambat jika derajat polinomialnya tinggi. *Kernel trick* memungkinkan kita mendapatkan hasil yang sama seolah-olah kita menambahkan banyak fitur polinomial, tanpa benar-benar harus menambahkannya. Ini membuat SVM sangat efisien.

In [ ]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler

# Contoh menggunakan dataset moons
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=100, noise=0.15, random_state=42)

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])
poly_kernel_svm_clf.fit(X, y)

### Gaussian RBF Kernel

Teknik lain untuk menangani masalah non-linear adalah dengan menambahkan fitur berdasarkan fungsi kesamaan (*similarity function*), yang mengukur seberapa mirip setiap *instance* dengan *landmark* tertentu. **Gaussian Radial Basis Function (RBF)** adalah fungsi kesamaan yang populer.

Sama seperti *polynomial kernel*, *kernel trick* juga dapat diterapkan di sini, memungkinkan kita mendapatkan hasil yang sama seperti menambahkan banyak fitur kesamaan tanpa biaya komputasi yang besar.

*Hyperparameter* `gamma` pada RBF kernel bertindak seperti *regularization*: jika model *overfitting*, turunkan `gamma`; jika *underfitting*, naikkan `gamma`.

In [ ]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])
rbf_kernel_svm_clf.fit(X, y)

## SVM Regression

SVM juga dapat digunakan untuk regresi. Triknya adalah dengan membalik tujuannya: alih-alih mencoba memasang *margin* terlebar di antara dua kelas, SVM Regression mencoba memasukkan sebanyak mungkin *instance* ke dalam *margin* sambil membatasi *margin violations*.

Lebar *margin* dikendalikan oleh *hyperparameter* `epsilon` ($\epsilon$).

In [ ]:
from sklearn.svm import LinearSVR

# Membuat data acak linear
np.random.seed(42)
m = 50
X = 2 * np.random.rand(m, 1)
y = (4 + 3 * X + np.random.randn(m, 1)).ravel()

svm_reg = LinearSVR(epsilon=1.5, random_state=42)
svm_reg.fit(X, y)